# Tutorial on descriptor 

In [1]:
# %load datadescriptor.py
"""A typical data descriptor.
"""

from __future__ import print_function

class DataDescriptor(object):
    """A simple descriptor.
    """
    def __init__(self):
        self.value = 0
    def __get__(self, instance, cls):
        print('data descriptor __get__')
        return self.value
    def __set__(self, instance, value):
        print('data descriptor __set__')
        try:
            self.value = value.upper()
        except AttributeError:
            self.value = value
    def __delete__(self, instance):
        print("Don't like to be deleted." )


Cool so in this tutorial we are going to learn about the data descriptor in python. These are very important but many at times are ignored.

In [2]:
class A(object):
    attr = DataDescriptor()
    pass

#### why to import objects??

So you wont find any difference as such in python3 because it is automatically imported. So main reason is to escape the old style 

So to know the difference just do this in python2
```python
class A():
    attr = DataDescriptor()
    pass

A.__dict__['attr'] = 100
```
No error  
now just add object you will get an error, so the point is adding object includes new functionality.

In [3]:
a= A()
a.attr
#so when it attr is called the get descriptor is called.

data descriptor __get__


0

In [4]:
type(a).__dict__['attr'].__get__(a,type(a)) # so whats happening behind!!

data descriptor __get__


0

In [5]:
A.attr

data descriptor __get__


0

In [6]:
A.__dict__['attr'].__get__(None,A)

#earlier applied on instance, now applied on the class itself...

data descriptor __get__


0

In [7]:
A.__dict__['attr'] = 12

# so interesting thought of these dictionary are they are immutable.

TypeError: 'mappingproxy' object does not support item assignment

In [8]:
a.attr = 5

data descriptor __set__


In [9]:
a.attr = 'hello'

data descriptor __set__


In [10]:
a.attr
#its happening because we have modified the get by adding upper case.

data descriptor __get__


'HELLO'

In [11]:
a.x = 100

In [12]:
a.__dict__

{'x': 100}

In [16]:
class NonDataDescriptor(object):
    """A simple descriptor.
    """
    def __init__(self):
        self.value = 0
    def __get__(self, instance, cls):
        print('data descriptor __get__')
        return self.value + 10
    pass

In [17]:
class A(object):
    attr = NonDataDescriptor()
    pass

In [18]:
a = A()
a.attr

data descriptor __get__


10

In [19]:
a.attr = 50

In [20]:
a.__dict__

{'attr': 50}

In [21]:
a.attr

50

So from here I want you to clarify that when attr is called the get function of the NonDescriptorClass was called. But when we assign the value, the normal set method is called. And when we try to get the value the normal get method value is returned. Thats the reason you can see the attr now in the dict of a.!!

#### ```__getattr__ and __getattribute__``` 

In [28]:
class Overridden(object):
    attr = DataDescriptor()
    def __getattribute__(self,name):
        print (name + " -> no way")
        pass
    pass


In [29]:
o = Overridden()
o.attr

attr -> no way


In [30]:
def fun():
    pass

fun.__get__

#its normal as it calls for data descriptor

<method-wrapper '__get__' of function object at 0x7ff2eeffd620>

In [34]:
class C(object):
    def meth():
        pass
    pass

c = C()
c.meth

#return bound method same function on python2 returns unbound method

<bound method C.meth of <__main__.C object at 0x7ff2fc053be0>>

In [35]:
type(c).__dict__['meth'].__get__(c,type(c))

<bound method C.meth of <__main__.C object at 0x7ff2fc053be0>>

In [40]:
# %load deco_meta/descriptors/class_storage.py

"""A descriptor works only in a class.

Storing attribute data directly in a descriptor
means sharing between instances.
"""

from __future__ import print_function


class DescriptorClassStorage(object):
    """Descriptor storing data in class."""

    def __init__(self, default=None):
        self.value = default

    def __get__(self, instance, owner):
        return self.value

    def __set__(self, instance, value):
        self.value = value


if __name__ == '__main__':
    class StoreClass(object):
        """All instances will share `attr`.
        """
        attr = DescriptorClassStorage(10)

    store1 = StoreClass()
    store2 = StoreClass()
    print('store1', store1.attr)
    print('store2', store2.attr)
    print('Setting store1 only.')
    store1.attr = 100
    print('store1', store1.attr) #
    print('store2', store2.attr) # both returns the same value.


store1 10
store2 10
Setting store1 only.
store1 100
store2 100


In [7]:
# %load deco_meta/descriptors/weakkeydict_storage.py

"""A descriptor works only in a class.

We can store a different value for each instance in a dictionary
in the descriptor.
"""

from weakref import WeakKeyDictionary


class DescriptorWeakKeyDictStorage(object):
    """Descriptor that stores attribute data in instances.
    """
    _hidden = WeakKeyDictionary()

    def __init__(self, default=None):
        self.default = default

    def __get__(self, instance, owner):
        return DescriptorWeakKeyDictStorage._hidden.get(instance, self.default)

    def __set__(self, instance, value):
        DescriptorWeakKeyDictStorage._hidden[instance] = value


if __name__ == '__main__':
    class StoreInstance(object):
        """All instances have own `attr`.
        """
        attr = DescriptorWeakKeyDictStorage(10)

    store1 = StoreInstance()
    store2 = StoreInstance()
    print('store1', store1.attr)
    print('store2', store2.attr)
    store1.attr = 100
    store2.attr =10000
    print('store1', store1.attr)
    print('store2', store2.attr)
    print('_hidden:', list(DescriptorWeakKeyDictStorage._hidden.items()))
    del store1
    print('Deleted store1')
    store2.attr =1000
    print('_hidden:', list(DescriptorWeakKeyDictStorage._hidden.items()))
    print (store2.attr)



store1 10
store2 10
store1 100
store2 10000
_hidden: [(<__main__.StoreInstance object at 0x7fe3e472c5c0>, 10000), (<__main__.StoreInstance object at 0x7fe3e472c550>, 100)]
Deleted store1
_hidden: [(<__main__.StoreInstance object at 0x7fe3e472c5c0>, 1000)]
1000


So here we saw how to get different instances.

Now modifing the same class, keeping the hypothesis that hidden is nothing but a normal dict and working out.. let see... 

In [6]:
class DescriptorDictStorage(object):
    """Descriptor that stores attribute data in instances.
    """
    _hidden = {}

    def __init__(self, default=None):
        self.default = default

    def __get__(self, instance, owner):
        return DescriptorDictStorage._hidden.get(instance, self.default)

    def __set__(self, instance, value):
        DescriptorDictStorage._hidden[instance] = value
    
    def __del__(self, instance, value):
        print("I am in del section")

if __name__ == '__main__':
    class StoreInstance(object):
        """All instances have own `attr`.
        """
        attr = DescriptorDictStorage(10)

    store1 = StoreInstance()
    store2 = StoreInstance()
    print('store1', store1.attr)
    print('store2', store2.attr)
    store1.attr = 100
    store2.attr =10000
    print('store1', store1.attr)
    print('store2', store2.attr)
    print('_hidden:', list(DescriptorDictStorage._hidden.items()))
    del store1
    print('Deleted store1')
    print('_hidden:', list(DescriptorDictStorage._hidden.items()))
    del store2
    print('Deleted store2')
    print('_hidden:', list(DescriptorDictStorage._hidden.items()))

store1 10
store2 10
store1 100
store2 10000
_hidden: [(<__main__.StoreInstance object at 0x7fe3e472c588>, 100), (<__main__.StoreInstance object at 0x7fe3e472c5f8>, 10000)]
Deleted store1
_hidden: [(<__main__.StoreInstance object at 0x7fe3e472c588>, 100), (<__main__.StoreInstance object at 0x7fe3e472c5f8>, 10000)]
Deleted store2
_hidden: [(<__main__.StoreInstance object at 0x7fe3e472c588>, 100), (<__main__.StoreInstance object at 0x7fe3e472c5f8>, 10000)]


Nope it doesnt work!!

```Below is the professional way how to write the data descriptor and also also if any checking is also needed to be implemented.```

In [12]:
# %load deco_meta/descriptors/checked.py

"""Example for descriptor that checks conditions on attributes.
"""
from __future__ import print_function

from weakref import WeakKeyDictionary


class Checked(object):
    """Descriptor that checks with a user-supplied check function
    if an attribute is valid.
    """

    _hidden = WeakKeyDictionary()

    def __init__(self, checker=None, default=None):
        if checker:
            # checker must be a callable
            checker(default)
        self.checker = checker
        self.default = default

    def __get__(self, instance, owner):
        return Checked._hidden.get(instance, self.default)

    def __set__(self, instance, value):
        if self.checker:
            self.checker(value)
        Checked._hidden[instance] = value


if __name__ == '__main__':

    def is_int(value):
        """Check if value is an integer.
        """
        if not isinstance(value, int):
            raise ValueError('Int required {} found'.format(type(value)))

    class Restricted(object):
        """Use checked attributes.
        """
        attr1 = Checked(checker=is_int, default=10)
        attr2 = Checked(default=12.5)
        # Setting the default to float, `is_int` raises a `ValueError`.
        try:
            attr3 = Checked(checker=is_int, default=12.5)
        except ValueError:
            print('Cannot set default to float, must be int.')
            attr3 = Checked(checker=is_int, default=12)

    restricted = Restricted()
    print('attr1', restricted.attr1)
    restricted.attr1 = 100
    print('attr1', restricted.attr1)
    try:
        restricted.attr1 = 200.12
    except ValueError:
        print('Cannot set attr1 to float, must be int.')
        restricted.attr1 = 200


Cannot set default to float, must be int.
attr1 10
attr1 100
Cannot set attr1 to float, must be int.


## Write a code that takes positive numbers only.

In [20]:
class DataDescriptor(object):
    
    def __init__(self,check = None):
        self.value = 0
        self.check = check
        pass
    
    def __get__(self, instance, owner):
        return self.value
    
    def __set__(self, instance, value):
        if self.check:
            self.check(value)
            pass
        
        self.value = value
        pass
    
    pass

if __name__ == '__main__':
    
    def positive(value):
        if type(value) == str:
            raise ValueError("No string")
            pass
        
        elif value <=0:
            raise ValueError("No Value less than zero")
            pass
        
        else:
            return True
        
        pass
    
    class Test(object):
        
        attr = DataDescriptor(positive)
        pass
    
    x = Test()
    print (x.attr)
    x.attr = 10
    print (x.attr)
    x.attr = 1.2
    
    # x.attr = 'ew'
    # x.attr = -12
    # x.attr = -1.2
    
        
        

0
10
